In [ ]:
import numpy as np
import warnings
from IPython.display import Audio, display
from scipy.io import wavfile
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import windows
hann = signal.windows.hann

og_len = 5000 # 5 seconds
channels = 2  # Stereo audio
sr = 44100 # stream rate

audio_path = "../../resources/synth1.wav"

warnings.simplefilter("ignore", wavfile.WavFileWarning)
sr_loaded, y = wavfile.read(audio_path)

# Convert to float32 and shape to (channels, samples)
waveform = y.T.astype(np.float32) / np.max(np.abs(y))  # normalize
num_samples = waveform.shape[1]

print(f"Original:")
display(Audio(waveform, rate=sr))

In [ ]:
# === PARAMETERS ===
grain_size_ms = 250
spray_ms = 10
feedback = 0.35
mix = 1.0

sr = sr_loaded
grain_size = int(sr * grain_size_ms/1000)
launch_interval = int(grain_size * 0.25)  # 75% overlap
spray = int(sr * spray_ms/1000)
window = hann(grain_size)

# === GRAIN DELAY ===
def grain_delay_stereo(x):
    ch, N = x.shape

    # output + delay accumulation buffer
    delay = np.zeros_like(x, dtype=np.float32)

    base_delay_in_samples = int(0.25 * sr)   # ~250ms delay

    # launch grains at overlapping intervals
    for n in range(0, N - grain_size, launch_interval):
        # --- input-side grain start ---
        start = n                      # where grain is "launched"
        end = start + grain_size

        # jitter the READ head
        gstart = np.clip(
            start + np.random.randint(-spray, spray),
            0, N - grain_size
        )
        gend = gstart + grain_size

        grain = x[:, gstart:gend]      # (ch, grain_size)
        windowed = grain * window      # apply Hann

        # --- output-side write position ---
        out_start = start + base_delay_in_samples
        out_end = out_start + grain_size

        # clamp output region to avoid overflow
        if out_start >= delay.shape[1]:
            break  # no more room

        if out_end > delay.shape[1]:
            usable = delay.shape[1] - out_start
            windowed = windowed[:, :usable]   # trim grain
            out_end = delay.shape[1]
        else:
            usable = grain_size

        # accumulate (DON'T replace)
        delay[:, out_start:out_end] += windowed
        delay[:, out_start:out_end] += feedback * delay[:, out_start:out_end]

    # final dry/wet mix
    out = (1.0 - mix) * x + mix * delay
    return out.astype(np.float32)

# === PROCESS ===
processed = grain_delay_stereo(waveform)
print("Grains:")
display(Audio(processed, rate=sr))